In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
from datetime import datetime
import numpy as np

In [3]:
# Check Station identifiers and locate Plant City, FL (USC00087205)
stn_ids = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-stations.txt', header = None, infer_nrows = 1000)
stn_ids.columns = ['ID', 'LAT', 'LON', 'ELEV', 'UKN', 'NAME', 'GSN', 'WBAN']
stn_ids

,ID,LAT,LON,ELEV,UKN,NAME,GSN,WBAN
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,NaN,ST JOHNS,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,NaN,SHARJAH INTER. AIRP,GSN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,NaN,DUBAI INTL,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,NaN,ABU DHABI INTL,NaN,41217.0
...,...,...,...,...,...,...,...,...
129653,ZI000067969,-21.0500,29.3670,861.0,NaN,WEST NICHOLSON,NaN,67969.0
129654,ZI000067975,-20.0670,30.8670,1095.0,NaN,MASVINGO,NaN,67975.0
129655,ZI000067977,-21.0170,31.5830,430.0,NaN,BUFFALO RANGE,NaN,67977.0
129656,ZI000067983,-20.2000,32.6160,1132.0,NaN,CHIPINGE,GSN,67983.0


In [4]:
# Open station inventory file to view variables
periods = pd.read_fwf('http://noaa-ghcn-pds.s3.amazonaws.com/ghcnd-inventory.txt', header = None, infer_nrows = 1000)
periods.columns = ['ID', 'LAT', 'LON', 'ELEM', 'TiMIN', 'TiMAX']
periods

,ID,LAT,LON,ELEM,TiMIN,TiMAX
0,ACW00011604,17.1167,-61.7833,TMAX,1949,1949
1,ACW00011604,17.1167,-61.7833,TMIN,1949,1949
2,ACW00011604,17.1167,-61.7833,PRCP,1949,1949
3,ACW00011604,17.1167,-61.7833,SNOW,1949,1949
4,ACW00011604,17.1167,-61.7833,SNWD,1949,1949
...,...,...,...,...,...,...
767508,ZI000067983,-20.2000,32.6160,PRCP,1951,2025
767509,ZI000067983,-20.2000,32.6160,TAVG,1962,2025
767510,ZI000067991,-22.2170,30.0000,TMAX,1951,1990
767511,ZI000067991,-22.2170,30.0000,TMIN,1951,1990


In [5]:
# Merge both tables into one large table using station ID
merged_stns = pd.merge(stn_ids, periods, how = 'left', left_on = 'ID', right_on = 'ID')
merged_stns

,ID,LAT_x,LON_x,ELEV,UKN,NAME,GSN,WBAN,LAT_y,LON_y,ELEM,TiMIN,TiMAX
0,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,TMAX,1949.0,1949.0
1,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,TMIN,1949.0,1949.0
2,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,PRCP,1949.0,1949.0
3,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,SNOW,1949.0,1949.0
4,ACW00011604,17.1167,-61.7833,10.1,NaN,ST JOHNS COOLIDGE FLD,NaN,NaN,17.1167,-61.7833,SNWD,1949.0,1949.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
767572,ZI000067983,-20.2000,32.6160,1132.0,NaN,CHIPINGE,GSN,67983.0,-20.2000,32.6160,PRCP,1951.0,2025.0
767573,ZI000067983,-20.2000,32.6160,1132.0,NaN,CHIPINGE,GSN,67983.0,-20.2000,32.6160,TAVG,1962.0,2025.0
767574,ZI000067991,-22.2170,30.0000,457.0,NaN,BEITBRIDGE,NaN,67991.0,-22.2170,30.0000,TMAX,1951.0,1990.0
767575,ZI000067991,-22.2170,30.0000,457.0,NaN,BEITBRIDGE,NaN,67991.0,-22.2170,30.0000,TMIN,1951.0,1990.0


In [9]:
# Sort stations based on station status
merged_stns_v2 = merged_stns.sort_values('TiMAX', ascending = False)
merged_stns_v2

,ID,LAT_x,LON_x,ELEV,UKN,NAME,GSN,WBAN,LAT_y,LON_y,ELEM,TiMIN,TiMAX
383793,US1TXBST164,30.1019,-97.5530,166.4,TX,CEDAR CREEK 3.2 WNW,NaN,NaN,30.1019,-97.5530,DAPR,2025.0,2025.0
171208,SA000040430,24.5500,39.7000,636.0,NaN,AL-MADINAH,GSN,40430.0,24.5500,39.7000,TMIN,1958.0,2025.0
171210,SA000040430,24.5500,39.7000,636.0,NaN,AL-MADINAH,GSN,40430.0,24.5500,39.7000,TAVG,1958.0,2025.0
386684,US1TXCML272,29.8288,-98.3573,385.3,TX,SPRING BRANCH 4.9 SE,NaN,NaN,29.8288,-98.3573,MDPR,2024.0,2025.0
386683,US1TXCML272,29.8288,-98.3573,385.3,TX,SPRING BRANCH 4.9 SE,NaN,NaN,29.8288,-98.3573,DAPR,2024.0,2025.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
542832,USC00232832,39.1208,-92.5611,192.0,MO,FAYETTE 6ESE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
543997,USC00234652,39.1008,-94.3003,219.5,MO,LAKE CITY,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545193,USC00236421,38.7039,-92.9811,207.3,MO,OTTERVILLE 1E,NaN,NaN,NaN,NaN,NaN,NaN,NaN
546808,USC00238558,38.8700,-93.6225,216.4,MO,VALLEY CITY 1NW,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Changed station from USC00118740 to USW00014942
df = pd.read_csv("s3://noaa-ghcn-pds/csv/by_station/USC00087205.csv", storage_options = {"anon": True},
                 dtype = {'Q_FLAG': 'object', 'M_FLAG': 'object'}, parse_dates = ['DATE']).set_index('DATE')

df

/var/folders/3s/rr9y36hj24dd4b6bzsw0kr_w0000gn/T/ipykernel_83950/2734918766.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("s3://noaa-ghcn-pds/csv/by_station/USC00087205.csv", storage_options = {"anon": True},


,ID,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME
DATE,,,,,,,
1892-09-01,USC00087205,TMAX,322,NaN,NaN,6,NaN
1892-09-02,USC00087205,TMAX,317,NaN,NaN,6,NaN
1892-09-03,USC00087205,TMAX,317,NaN,NaN,6,NaN
1892-09-04,USC00087205,TMAX,322,NaN,NaN,6,NaN
1892-09-05,USC00087205,TMAX,333,NaN,NaN,6,NaN
...,...,...,...,...,...,...,...
2025-12-04,USC00087205,PRCP,0,NaN,NaN,H,1600.0
2025-12-05,USC00087205,PRCP,0,NaN,NaN,H,1600.0
2025-12-06,USC00087205,PRCP,0,NaN,NaN,H,1600.0


In [17]:
# Subset to 1991–2020
df = df.sort_index()
df_1991_2020 = df.loc["1991-01-01":"2020-12-31"].copy()
df_1991_2020.head()

,ID,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME
DATE,,,,,,,
1991-01-01,USC00087205,PRCP,0,NaN,NaN,0,1800.0
1991-01-01,USC00087205,TMIN,167,NaN,NaN,0,1800.0
1991-01-01,USC00087205,SNOW,0,P,NaN,0,NaN
1991-01-01,USC00087205,SNWD,0,NaN,NaN,0,NaN
1991-01-01,USC00087205,TOBS,217,NaN,NaN,0,1800.0


### Question 1: 
#### Strawberries are planted around October 1 and ready for harvest by the end of January. What is the mean risk of frost and freeze, defined as the mean number of days per month over the period 1991-2020 that the temperature has been observed to be less than or equal to 32 and 28 degrees Fahrenheit, respectively, that might damage the plants for each month during the October - January period? (25 points)

In [18]:
# Isolate TMIN for Frost/Freeze
tmin = df_1991_2020[df_1991_2020["ELEMENT"] == "TMIN"].copy()
tmin

,ID,ELEMENT,DATA_VALUE,M_FLAG,Q_FLAG,S_FLAG,OBS_TIME
DATE,,,,,,,
1991-01-01,USC00087205,TMIN,167,NaN,NaN,0,1800.0
1991-01-02,USC00087205,TMIN,183,NaN,NaN,0,1800.0
1991-01-03,USC00087205,TMIN,178,NaN,NaN,0,1800.0
1991-01-04,USC00087205,TMIN,178,NaN,NaN,0,1800.0
1991-01-05,USC00087205,TMIN,172,NaN,NaN,0,1800.0
...,...,...,...,...,...,...,...
2020-12-27,USC00087205,TMIN,11,NaN,NaN,7,1600.0
2020-12-28,USC00087205,TMIN,89,NaN,NaN,7,1600.0
2020-12-29,USC00087205,TMIN,117,NaN,NaN,7,1600.0


In [21]:
# Convert from C to F
tmin["TMIN_C"] = tmin["DATA_VALUE"] / 10 # Converts from standard format to tenths
tmin["TMIN_F"] = tmin["TMIN_C"] * 9/5 + 32 # Converts from C to F

In [23]:
# Verify
tmin["TMIN_C"]

DATE
1991-01-01    16.7
1991-01-02    18.3
1991-01-03    17.8
1991-01-04    17.8
1991-01-05    17.2
              ... 
2020-12-27     1.1
2020-12-28     8.9
2020-12-29    11.7
2020-12-30    12.8
2020-12-31    18.9
Name: TMIN_C, Length: 10682, dtype: float64

In [24]:
# Verify
tmin["TMIN_F"]

DATE
1991-01-01    62.06
1991-01-02    64.94
1991-01-03    64.04
1991-01-04    64.04
1991-01-05    62.96
              ...  
2020-12-27    33.98
2020-12-28    48.02
2020-12-29    53.06
2020-12-30    55.04
2020-12-31    66.02
Name: TMIN_F, Length: 10682, dtype: float64

In [25]:
# Determine Frost and Freeze
tmin["Frost"]  = (tmin["TMIN_F"] <= 32).astype(int)
tmin["Freeze"] = (tmin["TMIN_F"] <= 28).astype(int)

In [27]:
# Add Year and Month
tmin["Year"]  = tmin.index.year
tmin["Month"] = tmin.index.month

# Extract Season of importance (Oct-Jan)
tmin_season = tmin[tmin["Month"].isin([10, 11, 12, 1])]

In [28]:
# Group by Year and Month, count Frost and Freeze
monthly_total = (tmin_season.groupby(["Year", "Month"])[["Frost", "Freeze"]].sum().reset_index())
monthly_total

,Year,Month,Frost,Freeze
0,1991,1,0,0
1,1991,10,0,0
2,1991,11,0,0
3,1991,12,0,0
4,1992,1,3,0
...,...,...,...,...
115,2019,12,0,0
116,2020,1,1,0
117,2020,10,0,0
118,2020,11,0,0


In [30]:
# Determine average frost/freeze days per month
climo = (monthly_total.groupby("Month")[["Frost", "Freeze"]].mean())
month_labels = {10:"October", 11:"November", 12:"December", 1:"January"} # Rename for readability
climo.index = [month_labels[m] for m in climo.index]
climo.round(2)

,Frost,Freeze
January,1.87,0.50
October,0.00,0.00
November,0.03,0.00
December,0.60,0.17


### Question 2
####  To begin to explore the seasonal to sub-seasonal prediction of freeze events at this site, using code you adapt from Module 4, we're going to try to relate these cold events to the El Nino Southern Oscillation (ENSO).  You have a hypothesis that ENSO is related to seasonal prediction of freeze events, but you don't know which region to choose for calculating your anomalies.  The problem is that there are many ENSO indicies that represent forcing across the eastern and central Pacific: Which SST forcing region is most related to cold conditions in central Florida?

In [31]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cdsapi
from eofs.xarray import Eof
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from sklearn.decomposition import PCA
import xarray as xr

In [48]:
# Load ENSO file given
enso = pd.read_csv('https://www.cpc.ncep.noaa.gov/data/indices/sstoi.indices', sep = r"\s+", header = 0,
                   names = ["YR", "MON", "NINO1+2", "ANOM1+2", "NINO3", "ANOM3", "NINO4", "ANOM4", "NINO3.4", "ANOM3.4"])
enso

,YR,MON,NINO1+2,ANOM1+2,NINO3,ANOM3,NINO4,ANOM4,NINO3.4,ANOM3.4
0,1982,1,24.28,-0.24,25.84,0.17,28.01,-0.21,26.65,0.08
1,1982,2,25.38,-0.72,26.26,-0.11,27.99,-0.11,26.54,-0.20
2,1982,3,25.22,-1.38,26.92,-0.25,28.18,-0.05,27.09,-0.14
3,1982,4,24.57,-1.16,27.52,-0.05,28.61,0.10,27.83,0.02
4,1982,5,24.00,-0.62,27.70,0.49,29.19,0.40,28.37,0.49
...,...,...,...,...,...,...,...,...,...,...
522,2025,7,22.29,0.46,25.92,0.04,28.84,0.05,27.24,-0.06
523,2025,8,21.09,0.23,24.97,-0.24,28.63,-0.06,26.58,-0.33
524,2025,9,20.40,-0.18,24.60,-0.41,28.41,-0.27,26.32,-0.44
525,2025,10,20.83,-0.04,24.74,-0.35,28.36,-0.33,26.29,-0.48


In [50]:
# Copy the Freeze/Frost Dataframe
freeze_df = monthly_total.copy()

# Add Date and set index
freeze_df["Date"] = pd.to_datetime(dict(year = freeze_df["Year"], month = freeze_df["Month"], day = 15))
freeze_df = freeze_df.set_index("Date")[["Freeze"]]
freeze_df

,Freeze
Date,
1991-01-15,0
1991-10-15,0
1991-11-15,0
1991-12-15,0
1992-01-15,0
...,...
2019-12-15,0
2020-01-15,0
2020-10-15,0


In [51]:
# Add Date and set index
enso["Date"] = pd.to_datetime(dict(year = enso["YR"], month = enso["MON"], day = 15))
enso = enso.set_index("Date")

# Ensure dates align with Plant City data 
enso = enso.loc["1991-01-01":"2020-12-31"]
enso

,YR,MON,NINO1+2,ANOM1+2,NINO3,ANOM3,NINO4,ANOM4,NINO3.4,ANOM3.4
Date,,,,,,,,,,
1991-01-15,1991,1,23.73,-0.78,25.63,-0.05,28.62,0.40,26.89,0.33
1991-02-15,1991,2,25.70,-0.40,26.28,-0.10,28.40,0.30,26.87,0.14
1991-03-15,1991,3,26.32,-0.28,26.96,-0.20,28.38,0.15,27.16,-0.08
1991-04-15,1991,4,25.21,-0.52,27.39,-0.19,28.73,0.22,27.89,0.08
1991-05-15,1991,5,24.65,0.04,27.44,0.22,29.18,0.39,28.13,0.25
...,...,...,...,...,...,...,...,...,...,...
2020-08-15,2020,8,20.17,-0.69,24.79,-0.43,28.36,-0.34,26.38,-0.52
2020-09-15,2020,9,19.82,-0.76,24.21,-0.80,28.25,-0.43,26.12,-0.64
2020-10-15,2020,10,20.12,-0.76,24.19,-0.90,27.97,-0.71,25.64,-1.13


In [52]:
# Isolate Oct-Jan
freeze_df = freeze_df[freeze_df.index.month.isin([10, 11, 12, 1])]
enso_season = enso[enso.index.month.isin([10, 11, 12, 1])]

In [53]:
# Only concerned with anomalies
enso_anom = enso_season[["ANOM1+2", "ANOM3", "ANOM4", "ANOM3.4"]]

In [54]:
# Merge on the datetime index to create one dataframe for comparison
merged = freeze_df.join(enso_anom, how = "inner")
merged.head()

,Freeze,ANOM1+2,ANOM3,ANOM4,ANOM3.4
Date,,,,,
1991-01-15,0,-0.78,-0.05,0.40,0.33
1991-10-15,0,0.22,0.27,0.63,0.64
1991-11-15,0,0.29,0.73,0.44,0.89
1991-12-15,0,0.29,1.03,0.77,1.50
1992-01-15,0,0.07,1.18,0.58,1.67


In [62]:
# Pearson Correlation pairings
corrs = {
    "NINO1+2": merged["Freeze"].corr(merged["ANOM1+2"]),
    "NINO3": merged["Freeze"].corr(merged["ANOM3"]),
    "NINO4": merged["Freeze"].corr(merged["ANOM4"]),
    "NINO3.4": merged["Freeze"].corr(merged["ANOM3.4"]),
}
corrs = pd.Series(corrs) # Cleaner display
corrs

NINO1+2   -0.135629
NINO3     -0.122463
NINO4     -0.135148
NINO3.4   -0.112039
dtype: float64

In [64]:
print("Pearson r:")
print(corrs)

# Use Absolute Value to get the strongest correlation
print("\nBest ENSO index:", corrs.abs().sort_values(ascending = False).index[0]) 

Pearson r:
NINO1+2   -0.135629
NINO3     -0.122463
NINO4     -0.135148
NINO3.4   -0.112039
dtype: float64

Best ENSO index: NINO1+2
